In [ ]:
%load_ext autoreload
%autoreload 2

# Instamatic - basic data collection

Instamatic is a tool for automated electron diffraction data collection. It has interfaces for interfacing with the TEM (JEOL/TFS) and several cameras (Gatan/ASI Timepix/TVIPS).

This notebook shows some interactive examples of what basic data collection scripts may look like.

https://github.com/stefsmeets/instamatic

First, we must import and initialize the controller object, which defines the interface to the TEM and the camera. For this example, the microscope interaction is simulated.

In [ ]:
from instamatic import TEMController
from instamatic.formats import write_tiff

ctrl = TEMController.initialize()

## Discrete rotation electron diffraction

This is an example script to collect electron diffraction data with discrete rotation steps with steps of 10° from -60 to 60°. The data are saved in a tiff file.

In [ ]:
exposure = 0.5  # s
angles = range(-50, 51, 10)

for i, angle in enumerate(angles):
    ctrl.stage.set(a=angle)
    print(f"Frame {i} @ {ctrl.stage.a:.1f}°")
    img, h = ctrl.getImage(exposure=exposure)
    write_tiff(f"red_{i:04d}.tiff", img, header=h)
    

We can show the last image, but they are not very interesting, because they are simulated as noise :-)

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
plt.imshow(img)

## Continuous rotation electron diffraction

This is an example of what data collection would look like with a continuous rotation. The difference is that the target angle is set using `wait=False`. This means that the call is non-blocking, so that the script can continue. We will stop data collection once the angle has reached the target angle.

In [ ]:
exposure = 0.5
start_angle =  50 # degrees
end_angle   = -50 # degrees

ctrl.stage.set(a=start_angle)
ctrl.stage.set(a=end_angle, wait=False)

buffer = []
i = 0

while ctrl.stage.a != end_angle:
    i += 1
    img, h = ctrl.getImage(exposure=exposure)
    buffer.append((img, h))
    print(f"Frame {i} @ {ctrl.stage.a:.1f}°")

for img, h in buffer:
    write_tiff(f"cred_{i:04d}.tiff", img, header=h)
    

## Serial electron diffraction

In a serial electron diffraction experiment, data are collected for a large number of crystals. The coordinates can be obtained from images taken at low magnification, but in this example a grid of coordinates will be created.

In [ ]:
from instamatic.tools import prepare_grid_coordinates

nx, ny = 3, 3
stepsize = 1000 # nm -> µm

coords = prepare_grid_coordinates(nx=nx, ny=ny, stepsize=stepsize)
print(coords)    

We then loop over every coordinate and collect a diffraction pattern. We can then defocus the diffraction pattern to obtain an image of the crystal in the primary beam.

In [ ]:
exposure = 0.2  # s

ctrl.mode.set("diff")

for i, (x, y) in enumerate(coords):
    ctrl.stage.set(x=x, y=y)
    print(f"\nImage {i} @ Position: {ctrl.stage.xy}")

    img, h = ctrl.getImage(exposure=exposure)
    write_tiff(f"sed_diff_{i:04d}.tiff", img, header=h)

    ctrl.difffocus.defocus(offset=1500)
    
    img, h = ctrl.getImage(exposure=exposure)
    write_tiff(f"sed_image_{i:04d}.tiff", img, header=h)
    
    ctrl.difffocus.refocus()

Alternatively, Instamatic can read `.nav` files exported from SerialEM and loop over the items with the acquire flag set. We make use of the `acquire_at_items` functionality to make `instamatic` handle the stage movement for us. Here is a simple example of how to acquire a diffraction pattern at every position and write it to a tiff file.

In [ ]:
from instamatic.formats import write_tiff
from instamatic.serialem import read_nav_file

exposure = 0.2

markers = read_nav_file("nav.nav", acquire_only=True)

ctrl.mode.set("diff")

def acquire(ctrl):
    img, h = ctrl.getImage(exposure=exposure)
    write_tiff(f"diff_{i:04d}.tiff", img, header=h)

ctrl.acquire_at_items(markers, acquire=acquire)

`acquire_at_items` offers a lot of options to run custom experiments, by specifying routines to run before, after or during the experiment.

In [ ]:
def pre_acquire_1(ctrl):
    print("I run before the experiment")

def pre_acquire_2(ctrl):
    print("And so do I!")

def every_3(ctrl):
    print("I run every 3 cycles")

def every_5(ctrl):
    print("I run every 5 cycles")

def every_8(ctrl):
    print("I run every 8 cycles")

def acquire(ctrl):
    print("I'm actually collecting data...")
    img, h = ctrl.getImage(exposure=exposure)

def post_acquire(ctrl):
    print("I run after the experiment is finished (or interrupted)!")
    
every_n = {
    3: every_3,
    5: every_5,
    8: every_8
}

ctrl.acquire_at_items(markers, 
                      pre_acquire=(pre_acquire_1, pre_acquire_2),
                      acquire=acquire,
                      post_acquire=post_acquire,
                      every_n=every_n)